In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer

import os
import itertools
import json
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt

Matplotlib created a temporary config/cache directory at /tmp/pbs.2925400.pbsha.ib.sockeye/matplotlib-dijnb8mr because the default path (/home/gregdeon/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
# work around insane huggingface requirements
from datasets.utils.logging import set_verbosity_error
set_verbosity_error()

In [5]:
# from spotlight.datasets import *
from torch_spotlight.utils import *
from torch_spotlight.plot_utils import *

setupPlots()

In [9]:
os.environ['DATA_DIR'] = '/arc/project/st-kevinlb-1/gregdeon/spotlight/datasets'
os.environ['MODEL_DIR'] = '/arc/project/st-kevinlb-1/gregdeon/spotlight/models'

In [ ]:
model_dir = '/arc/project/st-kevinlb-1/gregdeon/spotlight/models'
amazon_model_path = os.path.join(model_dir, 'amazon')

In [11]:
data_dir = os.environ['DATA_DIR'] 
model_dir = os.environ['MODEL_DIR'] 
amazon_dir = os.path.join(data_dir, 'amazon')
amazon_model_path = os.path.join(model_dir, 'amazon')
squad_dir = os.path.join(data_dir, 'squad')
squad_model_path = os.path.join(model_dir, 'squad')

# Amazon reviews

## Dataset + tokenizer

In [12]:
dataset = load_from_disk(amazon_dir)['test']

In [13]:
dataset[1]

{'content': "Despite the fact that I have only played a small portion of the game, the music I heard (plus the connection to Chrono Trigger which was great as well) led me to purchase the soundtrack, and it remains one of my favorite albums. There is an incredible mix of fun, epic, and emotional songs. Those sad and beautiful tracks I especially like, as there's not too many of those kinds of songs in my other video game soundtracks. I must admit that one of the songs (Life-A Distant Promise) has brought tears to my eyes on many occasions.My one complaint about this soundtrack is that they use guitar fretting effects in many of the songs, which I find distracting. But even if those weren't included I would still consider the collection worth it.",
 'label': 1,
 'title': "One of the best game music soundtracks - for a game I didn't really play"}

In [14]:
num_examples = 20000
amazon_labels = [dataset[i]['label'] for i in range(num_examples)]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english", 
    cache_dir=amazon_model_path, 
    do_lower_case=True, 
    do_basic_tokenize=True,
    local_files_only=True
)

In [11]:
unique_tokens = []
for i in tqdm(range(num_examples)):
    tokenized = tokenizer(dataset[i]['content'], padding=True, return_tensors='pt')
    unique_tokens.append(torch.unique(tokenized['input_ids']))

100%|██████████| 20000/20000 [00:09<00:00, 2015.61it/s]


## Model outputs

In [13]:
embeddings, outputs, losses = loadInferenceResults('inference_results/amazon_train_sst_20k.pkl')

In [14]:
loss_idx = losses.argsort().tolist()[::-1]
for idx in loss_idx[:10]:
    example = dataset[idx]
    display(example['content'])
    display(example['label'])
    display(losses[idx])
    print()

"This production is so awesome we can't stop watching it over and over. It is one of the finest vocal performances I have ever seen."

0

9.071007

'Very nice peridot necklace. I bought the 3-piece set for a birthday gift, and I was proud to give it as a gift.'

0

...

0

8.728806

'The background music is not clear and the CD was a waste of money. One star is too high.'

1

8.652259

## Spotlights

In [15]:
spotlights = {}
for count in np.arange(1, 6):
    spotlights[count] = loadSpotlightResults(f'spotlight_results/amazon_train_sst_20k_0.05_{count}.pkl')

In [24]:
# Example of high-weight reviews
spotlight = spotlights[3]
weight_idx = spotlight[0].argsort().tolist()[::-1]
for idx in weight_idx[:50]:
    example = dataset[idx]
    display(idx)
    display(example['content'])
    display(example['label'])
    display(losses[idx])
    display(outputs[idx])
    print()

5253

'I gave one star because i could not play game.Im glad i only paid $6.95.Thair is a way to get the game to play on windows 7,you have to set you boot advanced options to 1 core every time you want to play game.'

0

0.0050201556

0.0050075655

...

11176

"Haven't really been able to use as hoped. Have tried on several occasions but before I really get started the nozzle clogs. Using coal slag as the media which is pretty darn fine...but the second you stop the flow of material with stop it backs up and plugs. Air flow settings are key and just haven't found the settings to make it work yet."

0

0.004660577

0.004649722

In [19]:
def getCommonTokens(weights, smoothing=0.01, print_first=50):
    token_frequencies = defaultdict(float)
    token_frequencies_spotlight = defaultdict(float)

    weights_uniform = np.full_like(weights, 1 / len(weights))

    num_examples = len(weights)
    for i in tqdm(range(num_examples)):
        for token in unique_tokens[i]:
            token_frequencies[token.item()] += weights_uniform[i]
            token_frequencies_spotlight[token.item()] += weights[i]
            
    token_lrs = {k: (smoothing+token_frequencies_spotlight[k]) / (smoothing+token_frequencies[k]) for k in token_frequencies}
    tokens_sorted = list(map(lambda x: x[0], sorted(token_lrs.items(), key=lambda x: x[1])[::-1]))
    for i, (token) in enumerate(tokens_sorted[:print_first]):
        print('%15s %.4f %.4f %4.2f' % (tokenizer.decode(token), token_frequencies[token], token_frequencies_spotlight[token], token_lrs[token]))

In [20]:
getCommonTokens(spotlights[1][0], 0.005, 20)

100%|██████████| 20000/20000 [00:03<00:00, 6419.12it/s]


            que 0.0028 0.0112 2.08
            est 0.0023 0.0083 1.83
            las 0.0016 0.0069 1.80
           como 0.0011 0.0059 1.78
           ##as 0.0057 0.0135 1.72
              y 0.0071 0.0153 1.68
            tod 0.0011 0.0053 1.67
             si 0.0045 0.0109 1.66
             la 0.0080 0.0162 1.63
            dry 0.0088 0.0174 1.63
            por 0.0017 0.0059 1.62
             es 0.0060 0.0128 1.61
             ot 0.0015 0.0054 1.59
        regular 0.0070 0.0138 1.57
           ##lu 0.0034 0.0083 1.57
        however 0.0503 0.0810 1.55
           ##sa 0.0090 0.0168 1.55
           tiny 0.0038 0.0086 1.54
           para 0.0029 0.0071 1.53
       returned 0.0091 0.0166 1.53


In [21]:
getCommonTokens(spotlights[2][0], 0.005, 20)

100%|██████████| 20000/20000 [00:03<00:00, 6399.25it/s]


         bigger 0.0035 0.0090 1.65
          super 0.0104 0.0203 1.64
           hang 0.0028 0.0075 1.60
         prefer 0.0057 0.0120 1.58
        killing 0.0028 0.0074 1.58
            job 0.0239 0.0405 1.57
       discover 0.0029 0.0075 1.57
           slip 0.0026 0.0070 1.57
            rev 0.0039 0.0089 1.56
           easy 0.0415 0.0676 1.56
        wearing 0.0035 0.0083 1.56
          ##ves 0.0019 0.0058 1.56
         ##ling 0.0059 0.0119 1.55
             bt 0.0013 0.0048 1.54
           heat 0.0049 0.0103 1.54
           thus 0.0037 0.0084 1.54
           trip 0.0090 0.0165 1.54
            ##x 0.0118 0.0209 1.54
          handy 0.0021 0.0059 1.53
         source 0.0043 0.0092 1.53


In [22]:
getCommonTokens(spotlights[3][0], 0.005, 20)

100%|██████████| 20000/20000 [00:03<00:00, 6372.25it/s]

        problem 0.0279 0.0531 1.76
        ##point 0.0113 0.0236 1.76
      returning 0.0048 0.0120 1.73
          ##ssa 0.0082 0.0173 1.69
          ##und 0.0071 0.0153 1.67
              $ 0.0234 0.0420 1.66
         hoping 0.0092 0.0182 1.63
       returned 0.0091 0.0180 1.63
  unfortunately 0.0158 0.0280 1.58
           okay 0.0088 0.0167 1.57
         ##ware 0.0072 0.0142 1.56
            ##0 0.0057 0.0118 1.56
            box 0.0181 0.0310 1.55
         unless 0.0141 0.0244 1.54
          sadly 0.0051 0.0103 1.52
             ok 0.0174 0.0291 1.52
          maybe 0.0297 0.0477 1.52
        however 0.0503 0.0788 1.52
           half 0.0210 0.0344 1.51
           warn 0.0022 0.0060 1.51


In [144]:
loss_weights = (losses > 6.13)
loss_weights = high_loss_weights / high_loss_weights.sum()
getCommonTokens(loss_weights, 0.005, 20)

100%|██████████| 20000/20000 [00:03<00:00, 5456.58it/s]

          ##ven 0.0025 0.0174 2.98
            sen 0.0033 0.0174 2.71
             64 0.0010 0.0099 2.51
         length 0.0052 0.0199 2.42
       outdated 0.0022 0.0124 2.42
         potter 0.0022 0.0124 2.40
         bubble 0.0012 0.0099 2.39
              = 0.0024 0.0124 2.34
            rom 0.0014 0.0099 2.33
        contact 0.0037 0.0149 2.30
 cinematography 0.0026 0.0124 2.29
      adjusting 0.0006 0.0074 2.22
      functions 0.0018 0.0099 2.19
          stock 0.0042 0.0149 2.16
         versus 0.0008 0.0074 2.15
             bu 0.0032 0.0124 2.14
            fix 0.0044 0.0149 2.12
      confirmed 0.0010 0.0074 2.09
           wars 0.0021 0.0099 2.09
          ##nes 0.0010 0.0074 2.07


# SQuAD

## Dataset + tokenizer

In [260]:
dataset = load_from_disk(squad_dir)['validation']

In [261]:
categories = list(set(dataset['title']))

In [262]:
def filter_short_examples(example):
    example_length = len(tokenizer(
        example["question"],
        example["context"],
    )['input_ids'])
    return example_length < 384
short_dataset = dataset.filter(filter_short_examples)

Token indices sequence length is longer than the specified maximum sequence length for this model (629 > 512). Running this sequence through the model will result in indexing errors


In [263]:
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-distilled-squad", 
    cache_dir=squad_model_path, 
    local_files_only=True
)

In [264]:
unique_tokens = []
for i in tqdm(range(len(short_dataset))):
    example = short_dataset[i]
    tokenized = tokenizer(example["question"], example["context"], return_tensors='pt')
    unique_tokens.append(torch.unique(tokenized['input_ids']))

100%|██████████| 10386/10386 [00:11<00:00, 934.09it/s]


## Model outputs

In [265]:
embeddings, outputs, losses = loadInferenceResults('inference_results/squad_val_bert.pkl')

In [266]:
# Examples of high-loss questions
loss_idx = losses.argsort().tolist()[::-1]
for idx in loss_idx[0:10]:
    example = short_dataset[idx]
#     display(example['title'])
#     display(example['context'])
    display(example['question'])
    display(example['answers']['text'])
    display(losses[idx])
    print()

'Who disliked the affiliate program?'

['passed',
 'Several University of Chicago professors',
 'Several University of Chicago professors']

15.388167

'In which year were the North and South Courts opened?'

['Secretariat Wing', 'June 1862', 'June 1862']

...

['sending an email', 'email', 'Supreme Court case', 'broadcasting']

12.751001

'How has civil disobedience evolved in current times?'

['code-word describing the activities of muggers, arsonists, draft evaders',
 'utterly debased',
 'become utterly debased',
 'become utterly debased',
 'become utterly debased']

12.487267

## Spotlights

In [269]:
spotlights = {}
for count in np.arange(1, 6):
    spotlights[count] = loadSpotlightResults(f'spotlight_results/squad_val_bert_0.05_spherical_{count}.pkl')

### Common tokens

In [271]:
getCommonTokens(spotlights[1][0], 0.005, 20)

100%|██████████| 10386/10386 [00:02<00:00, 3513.94it/s]


         packet 0.0048 0.0114 1.67
     networking 0.0033 0.0088 1.67
            pad 0.0041 0.0100 1.64
        packets 0.0032 0.0082 1.62
      switching 0.0047 0.0105 1.60
  communication 0.0066 0.0127 1.52
          ##bed 0.0181 0.0290 1.47
       messages 0.0026 0.0062 1.47
       ignition 0.0016 0.0047 1.46
          ##lee 0.0032 0.0068 1.45
        circuit 0.0039 0.0079 1.44
      bandwidth 0.0013 0.0041 1.44
        ##ience 0.0184 0.0285 1.43
          ##ncy 0.0020 0.0051 1.43
       paradigm 0.0007 0.0031 1.43
           pure 0.0053 0.0097 1.42
      protocols 0.0016 0.0044 1.42
          alice 0.0025 0.0056 1.41
      energetic 0.0013 0.0038 1.41
         inputs 0.0026 0.0057 1.40


In [272]:
getCommonTokens(spotlights[2][0], 0.005, 20)

100%|██████████| 10386/10386 [00:02<00:00, 3546.26it/s]

          ##bed 0.0181 0.0300 1.52
            why 0.0238 0.0380 1.49
         ##ient 0.0086 0.0151 1.48
        ##ience 0.0184 0.0295 1.48
           know 0.0065 0.0113 1.41
           ##so 0.0236 0.0351 1.40
         arrest 0.0052 0.0093 1.40
          ##lor 0.0292 0.0427 1.40
     collective 0.0034 0.0065 1.38
          ##las 0.0271 0.0390 1.37
             ch 0.0285 0.0410 1.37
         packet 0.0048 0.0085 1.37
          ##lak 0.0089 0.0136 1.34
       membrane 0.0097 0.0146 1.33
           wage 0.0042 0.0073 1.33
        happens 0.0040 0.0070 1.33
          might 0.0167 0.0236 1.32
     protesters 0.0015 0.0036 1.32
           thor 0.0056 0.0090 1.32
         guilty 0.0034 0.0060 1.32


In [273]:
getCommonTokens(spotlights[3][0], 0.005, 20)

100%|██████████| 10386/10386 [00:02<00:00, 3587.59it/s]

          ##bed 0.0181 0.0257 1.33
         packet 0.0048 0.0080 1.32
            why 0.0238 0.0327 1.31
        ##ience 0.0184 0.0256 1.31
         ##ient 0.0086 0.0123 1.28
        packets 0.0032 0.0054 1.27
           ##so 0.0236 0.0306 1.24
     punishment 0.0066 0.0093 1.23
           know 0.0065 0.0092 1.23
          ##lor 0.0292 0.0365 1.21
         arrest 0.0052 0.0074 1.21
       messages 0.0026 0.0042 1.21
             ch 0.0285 0.0354 1.21
      switching 0.0047 0.0067 1.20
        circuit 0.0039 0.0058 1.20
     collective 0.0034 0.0050 1.20
          ##las 0.0271 0.0332 1.19
          wages 0.0072 0.0096 1.19
         guilty 0.0034 0.0050 1.19
     networking 0.0033 0.0048 1.19


In [274]:
# getCommonTokens(spotlights[4][0], 0.005, 20)

In [275]:
# getCommonTokens(spotlights[5][0], 0.005, 20)

In [229]:
np.quantile(losses, 0.98)

7.583634710311874

In [230]:
loss_weights = (losses > 7.58)
loss_weights = loss_weights / loss_weights.sum()
getCommonTokens(loss_weights, 0.005, 20)

100%|██████████| 10386/10386 [00:02<00:00, 4065.87it/s]


          sacks 0.0079 0.0529 4.49
            ##½ 0.0065 0.0433 4.18
        tackles 0.0065 0.0433 4.18
       confused 0.0020 0.0240 4.14
          ##bed 0.0181 0.0865 3.96
        ##ience 0.0184 0.0865 3.91
          yards 0.0161 0.0769 3.89
       behavior 0.0052 0.0337 3.79
     touchdowns 0.0065 0.0385 3.76
     defendants 0.0014 0.0192 3.76
     protesters 0.0015 0.0192 3.70
     cornerback 0.0041 0.0288 3.70
  interceptions 0.0081 0.0433 3.69
         fumble 0.0094 0.0481 3.68
         ##ling 0.0057 0.0337 3.62
         ##ient 0.0086 0.0433 3.56
      touchdown 0.0078 0.0385 3.40
          judge 0.0050 0.0288 3.38
         dallas 0.0037 0.0240 3.35
      defensive 0.0065 0.0337 3.35


### Topic distributions

In [276]:
def getTopicDistribution(weights, smoothing=0.01):
    topic_frequencies = defaultdict(float)
    topic_frequencies_spotlight = defaultdict(float)

    weights_uniform = np.full_like(weights, 1 / len(weights))

    num_examples = len(weights)
    for i in range(num_examples):
        example = short_dataset[i]
        category = example['title']
        topic_frequencies[category] += weights_uniform[i]
        topic_frequencies_spotlight[category] += weights[i]
        
    topic_ratios = {c: (smoothing + topic_frequencies_spotlight[c]) / (smoothing + topic_frequencies[c]) for c in topic_frequencies}

    categories_sorted = map(lambda x: x[0], sorted(topic_ratios.items(), key=lambda x: x[1], reverse=True))
    for category in categories_sorted:
        print('%.3f %.3f %.2f %s' % (topic_frequencies[category], topic_frequencies_spotlight[category], topic_ratios[category], category))

In [277]:
getTopicDistribution(spotlights[1][0], smoothing=0)

0.010 0.023 2.36 Packet_switching
0.019 0.031 1.61 Computational_complexity_theory
0.025 0.040 1.60 Teacher
0.019 0.030 1.57 Civil_disobedience
0.010 0.015 1.47 Intergovernmental_Panel_on_Climate_Change
0.017 0.023 1.40 French_and_Indian_War
0.024 0.031 1.29 Oxygen
0.021 0.026 1.25 Huguenot
0.020 0.025 1.24 Force
0.012 0.015 1.23 Pharmacy
0.010 0.012 1.19 Normans
0.012 0.014 1.18 Victoria_(Australia)
0.049 0.057 1.16 Nikola_Tesla
0.046 0.053 1.16 Martin_Luther
0.022 0.025 1.12 Kenya
0.030 0.033 1.10 Doctor_Who
0.010 0.011 1.08 Black_Death
0.028 0.031 1.08 Chloroplast
0.009 0.010 1.07 Jacksonville,_Florida
0.017 0.018 1.06 Southern_California
0.015 0.016 1.06 Prime_number
0.028 0.029 1.06 Economic_inequality
0.023 0.024 1.02 Genghis_Khan
0.011 0.011 1.01 Private_school
0.018 0.018 1.00 Scottish_Parliament
0.076 0.075 0.99 Super_Bowl_50
0.023 0.023 0.97 Apollo_program
0.018 0.016 0.92 Imperialism
0.010 0.009 0.90 Sky_(United_Kingdom)
0.021 0.019 0.87 Yuan_dynasty
0.028 0.024 0.87 Rhine
0

In [278]:
getTopicDistribution(spotlights[2][0], smoothing=0)

0.019 0.032 1.67 Civil_disobedience
0.010 0.015 1.57 Packet_switching
0.028 0.041 1.45 Chloroplast
0.017 0.022 1.35 French_and_Indian_War
0.015 0.020 1.34 Prime_number
0.012 0.016 1.33 Pharmacy
0.010 0.013 1.24 Black_Death
0.017 0.020 1.23 Ctenophora
0.023 0.027 1.18 Genghis_Khan
0.020 0.023 1.17 Force
0.024 0.027 1.15 Oxygen
0.028 0.032 1.15 Economic_inequality
0.019 0.022 1.14 Computational_complexity_theory
0.046 0.052 1.13 Martin_Luther
0.010 0.011 1.10 Intergovernmental_Panel_on_Climate_Change
0.022 0.024 1.09 Kenya
0.021 0.023 1.09 Yuan_dynasty
0.021 0.022 1.07 Huguenot
0.009 0.010 1.07 Jacksonville,_Florida
0.018 0.019 1.07 Imperialism
0.025 0.025 0.99 Teacher
0.028 0.028 0.99 Rhine
0.030 0.029 0.99 Doctor_Who
0.018 0.018 0.97 Scottish_Parliament
0.011 0.011 0.96 Geology
0.010 0.010 0.94 Sky_(United_Kingdom)
0.011 0.010 0.93 Private_school
0.023 0.021 0.93 Immune_system
0.049 0.046 0.93 Nikola_Tesla
0.023 0.021 0.92 Apollo_program
0.010 0.009 0.91 1973_oil_crisis
0.010 0.009 0.9

In [280]:
getTopicDistribution(spotlights[3][0], smoothing=0)

0.010 0.015 1.53 Packet_switching
0.019 0.027 1.43 Civil_disobedience
0.012 0.015 1.24 Pharmacy
0.028 0.035 1.24 Chloroplast
0.019 0.023 1.22 Computational_complexity_theory
0.017 0.020 1.22 French_and_Indian_War
0.025 0.030 1.19 Teacher
0.020 0.023 1.14 Force
0.015 0.017 1.14 Prime_number
0.010 0.011 1.13 Intergovernmental_Panel_on_Climate_Change
0.010 0.012 1.13 Black_Death
0.046 0.051 1.11 Martin_Luther
0.017 0.018 1.10 Ctenophora
0.024 0.026 1.10 Oxygen
0.028 0.030 1.09 Economic_inequality
0.023 0.025 1.06 Genghis_Khan
0.022 0.024 1.06 Kenya
0.021 0.022 1.04 Huguenot
0.009 0.010 1.03 Jacksonville,_Florida
0.049 0.050 1.02 Nikola_Tesla
0.021 0.022 1.01 Yuan_dynasty
0.018 0.018 1.01 Imperialism
0.030 0.030 1.00 Doctor_Who
0.018 0.018 1.00 Scottish_Parliament
0.010 0.010 0.99 Normans
0.012 0.012 0.96 Victoria_(Australia)
0.010 0.010 0.95 Sky_(United_Kingdom)
0.017 0.016 0.94 Southern_California
0.076 0.071 0.94 Super_Bowl_50
0.011 0.010 0.94 Private_school
0.028 0.026 0.93 Rhine
0.023

In [235]:
loss_weights = (losses > 7.58)
loss_weights = loss_weights / loss_weights.sum()
getTopicDistribution(loss_weights, 0)

0.019 0.091 4.82 Civil_disobedience
0.010 0.029 2.83 1973_oil_crisis
0.019 0.053 2.79 Computational_complexity_theory
0.076 0.168 2.21 Super_Bowl_50
0.018 0.038 2.15 Imperialism
0.012 0.024 2.01 Pharmacy
0.024 0.043 1.82 Oxygen
0.010 0.014 1.47 Packet_switching
0.010 0.014 1.44 Intergovernmental_Panel_on_Climate_Change
0.027 0.038 1.43 Victoria_and_Albert_Museum
0.017 0.024 1.42 University_of_Chicago
0.028 0.034 1.22 Economic_inequality
0.046 0.053 1.16 Martin_Luther
0.013 0.014 1.14 Fresno,_California
0.028 0.029 1.03 Rhine
0.028 0.029 1.02 Chloroplast
0.015 0.014 0.98 Prime_number
0.020 0.019 0.97 Force
0.025 0.024 0.96 Teacher
0.010 0.010 0.92 Normans
0.021 0.019 0.90 Yuan_dynasty
0.011 0.010 0.88 Private_school
0.023 0.019 0.84 Immune_system
0.049 0.038 0.78 Nikola_Tesla
0.021 0.014 0.69 Huguenot
0.017 0.010 0.58 French_and_Indian_War
0.018 0.010 0.53 United_Methodist_Church
0.009 0.005 0.52 Jacksonville,_Florida
0.009 0.005 0.51 Construction
0.030 0.014 0.48 Doctor_Who
0.010 0.005